In [100]:
import hashlib
import os
import tarfile
import zipfile
import requests

DATA_HUB = dict()
DATA_HUB['url'] = 'http://d2l-data.s3-accelerate.amazonaws.com/'

def download(name, cache_dir=os.path.join('..', 'data')):
    assert name in DATA_HUB, f"{name} does not exist in {DATA_HUB}"
    url, sha1_hash = DATA_HUB[name]
    os.makedirs(cache_dir, exist_ok=True)
    fname = os.path.join(cache_dir, url.split('/')[-1])
    if os.path.exists(fname):
        sha1 = hashlib.sha1()
        with open(fname, 'rb') as f:
            while True:
                data = f.read(1048576)
                if not data:
                    break
                sha1.update(data)
        if sha1.hexdigest() == sha1_hash:
            return fname
    print(f"Downloading {fname} from {url}...")
    r = requests.get(url, stream=True, verify=True)
    with open(fname, 'wb') as f:
        f.write(r.content)
    return fname

In [101]:
%matplotlib inline
import numpy as np
import pandas as pd
import torch
from torch import nn
from d2l import torch as d2l

DATA_HUB['kaggle_house_train'] = (DATA_HUB['url'] + 'kaggle_house_pred_train.csv',
                                    '585e9cc93e70b39160e7921475f9bcd7d31219ce')

DATA_HUB['kaggle_house_test'] = (DATA_HUB['url'] + 'kaggle_house_pred_test.csv',
                                    'fa19780a7b011d9b009e8bff8e99922a8ee2eb90')

train_data = pd.read_csv(download('kaggle_house_train'))
test_data = pd.read_csv(download('kaggle_house_test'))

print(train_data.shape)
print(test_data.shape)

(1460, 81)
(1459, 80)


In [102]:
print(train_data.iloc[0:4, [0, 1, 2, 3, -3, -2, -1]])

   Id  MSSubClass MSZoning  LotFrontage SaleType SaleCondition  SalePrice
0   1          60       RL         65.0       WD        Normal     208500
1   2          20       RL         80.0       WD        Normal     181500
2   3          60       RL         68.0       WD        Normal     223500
3   4          70       RL         60.0       WD       Abnorml     140000


In [103]:
test_data.iloc[0:4, [0, 1, 2, 3, -3, -2, -1]]

,Id,MSSubClass,MSZoning,LotFrontage,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,2010,WD,Normal
1,1462,20,RL,81.0,2010,WD,Normal
2,1463,60,RL,74.0,2010,WD,Normal
3,1464,60,RL,78.0,2010,WD,Normal


In [104]:
all_features = pd.concat((train_data.iloc[:, 1:-1], test_data.iloc[:, 1:]))

#### 处理数据：normalization

In [105]:
numeric_features = all_features.dtypes[all_features.dtypes != 'object'].index
all_features[numeric_features] = all_features[numeric_features].apply(
    lambda x: (x - x.mean()) / (x.std()))
all_features[numeric_features] = all_features[numeric_features].fillna(0)
all_features.shape

(2919, 79)

#### 处理离散值：one-hot encoding
- 将类别变量转化为多个二进制特征（0 或 1）。
- 每个类别对应一个新列，如果某一行属于该类别，则对应列的值为 1，否则为 0。

In [106]:
all_features = pd.get_dummies(all_features, dummy_na=True)
all_features.shape

(2919, 330)

In [107]:
n_train = train_data.shape[0]
all_features = all_features.replace({True: 1, False: 0})
train_features = torch.tensor(all_features[:n_train].values, dtype=torch.float32)
test_features = torch.tensor(all_features[n_train:].values, dtype=torch.float32)
train_labels = torch.tensor(train_data.SalePrice.values.reshape(-1, 1), dtype=torch.float32)

In [108]:
loss = nn.MSELoss()
in_features = train_features.shape[1]

def get_net():
    net = nn.Sequential(nn.Linear(in_features, 1))
    return net

#### 将labels与预测值取对数进行计算，计算相对误差

In [109]:
def log_rmse(net, features, labels):
    clipped_preds = torch.clamp(net(features), 1, float('inf'))
    rmse = torch.sqrt(loss(clipped_preds.log(), labels.log()))
    return rmse.item()

#### Adam优化器：自适应学习率？对学习率敏感程度较低

In [110]:
def train(net, train_features, train_labels, test_features, test_labels, num_epochs, learning_rate, weight_decay, batch_size):
    train_ls, test_ls = [], []
    train_iter = d2l.load_array((train_features, train_labels), batch_size)
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate, weight_decay=weight_decay)
    for epoch in range(num_epochs):
        for X, y in train_iter:
            optimizer.zero_grad()
            print(
               net(X).shape, y.shape,
               net(X), y
            )
            l = loss(net(X), y)
            l.backward()
            optimizer.step()
        train_ls.append(log_rmse(net, train_features, train_labels))
        if test_labels is not None:
            test_ls.append(log_rmse(net, test_features, test_labels))
    return train_ls, test_ls

In [111]:
def get_k_fold_data(k, i, X, y):
    # print(
    #     X,
    #     X.shape,
    #     y,
    #     y.shape,
    # )
    assert k > 1
    fold_size = X.shape[0] // k
    X_train, y_train = None, None
    for j in range(k):
        idx = slice(j * fold_size, (j + 1) * fold_size)
        X_part, y_part = X[idx, :], y[idx]
        # Validation set / training set
        if j == i:
            X_valid, y_valid = X_part, y_part
        elif X_train is None:
            X_train, y_train = X_part, y_part
        else:
            X_train = torch.cat([X_train, X_part], 0)
            y_train = torch.cat([y_train, y_part], 0)
    return X_train, y_train, X_valid, y_valid

In [112]:
def k_fold(k, x_train, y_train, num_epochs, learning_rate, weight_decay, batch_size):
    train_l_sum, valid_l_sum = 0, 0
    for i in range(k):
        data = get_k_fold_data(k, i, x_train, y_train)
        net = get_net()
        train_ls, valid_ls = train(net, *data, num_epochs, learning_rate, weight_decay, batch_size)
        train_l_sum += train_ls[-1]
        valid_l_sum += valid_ls[-1]
        if i == 0:
            d2l.plot(list(range(1, num_epochs + 1)), [train_ls, valid_ls], xlabel='epoch', ylabel='rmse', legend=['train', 'valid'], yscale='log')
        print(f'fold {i + 1}, train log rmse {float(train_ls[-1]):f}, valid log rmse {float(valid_ls[-1]):f}')
    return train_l_sum / k, valid_l_sum / k

In [113]:
k, num_epochs, lr, weight_decay, batch_size = 5, 100, 5, 0, 64
train_l, valid_l = k_fold(k, train_features, train_labels, num_epochs, lr, weight_decay, batch_size)
print(f'{k}-fold validation: avg train log rmse: {float(train_l):f}, avg valid log rmse: {float(valid_l):f}')

torch.Size([64, 1]) torch.Size([64, 1]) tensor([[ 1.1440e-01],
        [ 3.1720e-01],
        [ 2.9643e-01],
        [ 2.1254e-01],
        [ 3.0259e-01],
        [ 2.6419e-01],
        [ 5.2819e-01],
        [-2.8510e-02],
        [ 4.6230e-01],
        [ 5.2141e-01],
        [ 3.4633e-01],
        [ 2.9158e-01],
        [ 5.2011e-01],
        [ 1.3864e-01],
        [ 1.6908e-01],
        [ 2.4711e-01],
        [ 2.4363e-01],
        [ 3.3167e-01],
        [ 1.4154e-01],
        [ 4.1650e-02],
        [ 3.3852e-01],
        [ 3.0266e-01],
        [ 1.6244e-01],
        [ 3.4652e-01],
        [ 2.1554e-01],
        [ 1.3130e-01],
        [ 1.7395e-01],
        [-3.1487e-01],
        [ 3.2059e-01],
        [ 4.5249e-01],
        [ 4.7627e-01],
        [ 1.5221e-02],
        [ 3.4328e-01],
        [ 1.8205e-01],
        [ 2.9357e-01],
        [ 5.3456e-01],
        [-2.1172e-02],
        [ 1.3014e-01],
        [ 3.4318e-01],
        [ 1.4551e-01],
        [ 3.1552e-01],
        [ 2.3853e

RuntimeError: Boolean value of Tensor with more than one value is ambiguous